In [2]:
import os
import datetime
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
import seaborn as sns
%matplotlib inline
from __future__ import print_function
import numpy as np
import statsmodels.api as sm
from scipy import stats

test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

/Users/charleneyu1221/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/charleneyu1221/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
#print(test)
#sample.head()
traindata=train[['UserId', 'ProductId','Score' ]]
#traindata.index=traindata['UserId']
train.head()
len(train)

560804

In [30]:

%matplotlib inline
%config InlineBackend.figure_format='retina'
# import libraries
import numpy as np
import matplotlib as mp
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd

from importlib import reload
from datetime import datetime
from IPython.display import Image
from IPython.display import display_html
from IPython.display import display
from IPython.display import Math
from IPython.display import Latex
from IPython.display import HTML
print('')

In [24]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD()
X_transformed = svd.fit_transform(traindata)

ValueError: could not convert string to float: 'B00141SO7A'

In [12]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

from surprise import SVD
from surprise import dataset
from surprise import Reader
from surprise import evaluate, print_perf

df = pd.DataFrame.from_dict(traindata)

# You'll need to create a dummy reader
reader = Reader(line_format='user item rating', rating_scale=(1, 5))

# Also, a dummy Dataset class
class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, df, reader):

        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(df['UserId'], df['ProductId'], df['Score'])]
        self.reader=reader

data = MyDataset(df, reader)



In [13]:
# You should now be able to use the data object normally.
data.split(n_folds=3)
algo = SVD()
perf = evaluate(algo, data, measures=['RMSE', 'MAE'])
print_perf(perf)
#u,s,vt = np.linalg.svd(train,full_matrices=False)
#plt.plot(s)
#_ = plt.title('Singular Values of Spatial Like Matrix')

/Users/charleneyu1221/anaconda/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Users/charleneyu1221/anaconda/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: nan
MAE:  nan
------------
Fold 2
RMSE: nan
MAE:  nan


KeyboardInterrupt: 

In [38]:
traindata
#traindata.index=traindata['UserId']
#traindata=traindata.drop(['UserId'], axis=1)
#traindata.pivot(index='UserId', columns='ProductId', values='Score')
traindata1=traindata.array_split(traindata, 3)
len(traindata1)

AttributeError: 'DataFrame' object has no attribute 'array_split'

In [ ]:

traindata1=traindata[:300000]
new=traindata1.pivot(index='UserId', columns='ProductId', values='Score')

In [37]:
#stacked = traindata.stack()
#stacked
group=traindata.groupby(['UserId'])
len(group)
group1=traindata.groupby(['ProductId'])
group1.head()

,UserId,ProductId,Score
0,A3VZR9TPF2GERB,B000CQIDHY,5.0
1,A1B1QMGK8VYG80,B004YV80OE,4.0
2,A1AQ2W2R4SOVGN,B000G6MBX2,1.0
3,A2IVH1D3GLACL3,B0001GDC4O,5.0
4,AEOINN8F4D9DQ,B000EDG430,5.0
5,AUTBHG6070SL4,B000H26J7E,2.0
6,A1T1LLS0I1FFPR,B000UK3G2Y,5.0
7,A34ZQGFACNZRCL,B001E6KBJ4,2.0
8,A2OAC8Y7XGFTX7,B000EDROXO,1.0
9,A26732RC19B9YE,B000AXSG2Q,4.0


In [10]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')
data
# Use the famous SVD algorithm.
#algo = SVD()

# Run 5-fold cross-validation and print results.
#cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [14]:
data.isnull()


AttributeError: 'MyDataset' object has no attribute 'isnull'